<a href="https://colab.research.google.com/github/yingchengsun/Opinion-Spam-Detection/blob/data-prepossessing/Data_Prepossessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import json
from collections import OrderedDict
import urllib
from bs4 import BeautifulSoup
import pandas as pd
import binascii
import pickle
import base64
import re
import sys
import os.path
import time
import networkx as nx
import matplotlib.pyplot as plt

yelp="https://www.yelp.com"

In [0]:
def aggregate_reviews_with_biz():
  with open('/content/drive/My Drive/Yelp/yelp_dataset/business.json', 'r') as biz_inf:
    address = '/content/drive/My Drive/Yelp/Data/original/top1000/'
    for idx, line in enumerate(biz_inf):  
      if idx >= 1000:
        address = '/content/drive/My Drive/Yelp/Data/original/after1000/'
      print (idx, line)
      data_item = json.loads(line, object_pairs_hook=OrderedDict)
      biz_id = data_item['business_id']
      rev_count = data_item['review_count']
      with open(address + biz_id + '.json', 'w+') as outfile:
        with open('/content/drive/My Drive/Yelp/yelp_dataset/review.json', 'r') as rev_inf:
          for rev_line in rev_inf:
            rev_item = json.loads(rev_line, object_pairs_hook=OrderedDict) 
            if  biz_id == rev_item['business_id']:
              #print (rev_item['review_id']):
              json.dump(rev_item, outfile)
              outfile.write('\n')
              rev_count -= 1
              if rev_count == 0:
                break
              
      

In [0]:
def aggregate_reviews_with_biz():
  with open('/content/drive/My Drive/Yelp/yelp_dataset/business.json', 'r') as biz_inf:
    address = '/content/drive/My Drive/Yelp/Data/original/top1000/'
    biz_id = 'QXAEGFB4oINsVuTFxEYKFQ'
    for line in biz_inf:
      data_item = json.loads(line, object_pairs_hook=OrderedDict)
      if biz_id == data_item['business_id']:
        rev_count = data_item['review_count']
    with open(address + biz_id + '.json', 'w+') as outfile:
      with open('/content/drive/My Drive/Yelp/yelp_dataset/review.json', 'r') as rev_inf:
        for rev_line in rev_inf:
          rev_item = json.loads(rev_line, object_pairs_hook=OrderedDict) 
          if biz_id == rev_item['business_id']:
            print (rev_item['review_id'])
            json.dump(rev_item, outfile)
            outfile.write('\n')
            rev_count -= 1
            if rev_count == 0:
              break
aggregate_reviews_with_biz()
              

In [0]:
def filter_biz_closed():
  with open('/content/drive/My Drive/Yelp/Data/biz_closed.json', 'w+') as biz_closed:
    with open('/content/drive/My Drive/Yelp/yelp_dataset/business.json', 'r') as biz_inf:
      i=0
      for idx, line in enumerate(biz_inf):  
        if idx >= 1000:
          break
        data_item = json.loads(line, object_pairs_hook=OrderedDict)
        is_open = data_item['is_open']
        if is_open == 0:
            i+=1
            print (idx, line)
            biz_closed.write(line)
      print (i)

0 {"business_id":"1SWheh84yJXfytovILXOAQ","name":"Arizona Biltmore Golf Club","address":"2818 E Camino Acequia Drive","city":"Phoenix","state":"AZ","postal_code":"85016","latitude":33.5221425,"longitude":-112.0184807,"stars":3.0,"review_count":5,"is_open":0,"attributes":{"GoodForKids":"False"},"categories":"Golf, Active Life","hours":null}

8 {"business_id":"Y6iyemLX_oylRpnr38vgMA","name":"Vita Bella Fine Day Spa","address":"5940 W Union Hills Dr","city":"Glendale","state":"AZ","postal_code":"85308","latitude":33.6548146,"longitude":-112.1885676,"stars":5.0,"review_count":8,"is_open":0,"attributes":{"RestaurantsPriceRange2":"2","ByAppointmentOnly":"True","BusinessParking":"{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}","BusinessAcceptsCreditCards":"True","BikeParking":"False"},"categories":"Nail Salons, Beauty & Spas, Day Spas","hours":{"Tuesday":"12:0-18:0","Wednesday":"10:0-18:0","Thursday":"12:0-18:0","Friday":"10:0-17:0","Saturday":"10:0-14:0"}

In [0]:
google = "https://www.google.com/search?q="
def search_from_google():
  with open('/content/drive/My Drive/Yelp/yelp_dataset/business.json', 'r') as biz_inf:
    i=0
    for idx, line in enumerate(biz_inf):  
      if idx >= 10:
        break
      data_item = json.loads(line, object_pairs_hook=OrderedDict)
      is_open = data_item['is_open']
      if is_open == 0:
        print (line)
        name_city = data_item['name'] + data_item['city']
        g_q = google + name_city.replace(' ','+')
        page = urllib.request.urlopen(g_q)
        print (page)
        soup = BeautifulSoup(page)
        g_r= soup.findAll("div",{"class":"g"})   
        for r in g_r:
          if r.find('a').text.contains('CLOSED'):
            print (a['href'])

search_from_google()      
                                                  

In [0]:
def reviews_count_filtered():
  with open('/content/drive/My Drive/Yelp/Data/statistics/review_count_filtered.txt', 'w') as rev_fil:
    with open('/content/drive/My Drive/Yelp/Data/statistics/biz_closed.json', 'r') as biz:
      with open('/content/drive/My Drive/Yelp/Data/statistics/review_count2.txt', 'r') as rev:
        biz_id_dict= set()
        for b in biz:
          data_item = json.loads(b, object_pairs_hook=OrderedDict)
          if data_item['is_open'] == 0:
            biz_id_dict.add(data_item['business_id'])
        print (len(biz_id_dict))
        i=0
        for r in rev:
          if r.split()[0] not in biz_id_dict:
            rev_fil.write(r)
            i+=1
        print (i)

In [0]:
def reviews_count_filtered():
  recommended = not_recommended = 0
  with open('/content/drive/My Drive/Yelp/Data/statistics/review_count_filtered.txt', 'r') as rev_fil:
    i = 0
    low = 9999
    upper = 0
    for r in rev_fil:
      r_number = int(r.split()[1])
      n_r_number = int(r.split()[2])
      if n_r_number == 0:
        print (r)
      else:
        per = (1.0 * r_number) / n_r_number
        '''
        if per > upper:
          print (per)
          print (r)
          uppper = per
        '''  
        if per < low:
          print (per)
          print (r)
          low = per
      recommended += r_number
      not_recommended += n_r_number
      i += 1
    filtered = 1000 - i
  total = 1.0*(recommended + not_recommended)
  print ('recommended: '+str(recommended)+ ' ;percent: '+ str(recommended/total))
  print ('not_recommended: '+str(not_recommended)+ ' ;percent: '+ str(not_recommended/total))
  print ('total: '+str(total))
  print ('filtered business: '+str(filtered))
  print ('total business: '+str(i))
reviews_count_filtered()


5.071428571428571
QXAEGFB4oINsVuTFxEYKFQ 142 28

xvX2CttrVhyG2z1dFg_0xw 4 0

0.4
HhyxOkGAM07SRYtlQ4wMFQ 8 20

lxnuq9wJiwLOPJ4uZU2ljg 4 0

BsMdebN4nZySpGTfXr-7yg 7 0

C9oCPomVP0mtKa8z99E3gg 8 0

HVPcIcqiJkrpD36xZFGN6g 23 0

c-BELKj0SvNhBesQMf-bKw 3 0

cQ8FKduGfVskvi11NZXsAg 3 0

R3rss9fkfJxiOK6DueON3w 5 0

Y0dRlYjRNWckjvTGd55hzg 7 0

sV6kKpzneWwQ8cY8gQ8bgA 6 0

QHr_fc_TLnGfUEuDEa2jDg 7 0

X8mtoSxY8whtmbDlj0D3Aw 4 0

0.1388888888888889
80hHlCEz_1-vgAhmLvs3lQ 5 36

Kxbn0XLcCgQqmkdIkhuOeQ 11 0

N2G7ySm6R-jYrHvDAV2Ndw 9 0

DwsESfIDIsgQ4DTx9VNOsw 3 0

7_nq0GqNvc5CT0A3tVCpcg 4 0

xXdRivb3LjGCl8HiuUa22g 5 0

kGal388xLnmCwI4mw2TG8Q 7 0

cz5kBEZMHKyJ_nZXn-IRIA 4 0

EXr-oKAbgON2bYNinu6cyQ 6 0

v8zKjH1o_qAI4SBd3na7gg 14 0

cUesIMddiBM--8q5KxXu5w 4 0

3E0JDI0XMUoFztoEmX2D7w 3 0

TTVwcuc1Nz4ZSZH4wnbxKQ 6 0

sRLczTc6OHuttVSanRfHaw 7 0

4Rp1y8L_zecjHt6AvKtekg 3 0

ET2dIIR1qMThKyBEOL5BkA 15 0

lrYB3RSnr8OVrrqHhGMwPA 3 0

0.1
xOR85RicYj642O3_iJ7hgg 3 30

3kwigsDDg6DjYtKiqMr7mw 3 0

fFKbj19pBWMQosCn9qsTd

In [0]:
def pics_count():
 with open('/content/drive/My Drive/Yelp/Data/statistics/review_count_filtered.txt', 'r') as rev_fil:
   with open('/content/drive/My Drive/Yelp/Data/statistics/photo_count.txt', 'w') as photo_count:
    f = open('/content/drive/My Drive/Yelp/Data/statistics/user_id_review_count', 'rb')
    user_id = pickle.load(f)
    #print (user_id.keys())
    nr_user_id = []
    for r in rev_fil:
      b_id = r.split()[0]
      if os.path.exists('/content/drive/My Drive/Yelp/Data/scraped/' + b_id +'.json'):
        with open('/content/drive/My Drive/Yelp/Data/scraped/' + b_id +'.json', 'r') as biz:
          r_photo_count = nr__photo_count = 0
          r_review_count = nr_review_count = 0
          r_photo_is = nr_photo_is = 0
          r_photo_is_per = nr_photo_is_per = 0
          for line in biz:
            review = json.loads(line, object_pairs_hook=OrderedDict)
            photos = review['totalPhotos']
            label = review['label']
            if label == 1:
              r_review_count += 1
              if photos > 0:
                r_photo_is += 1
                r_photo_count += photos
            else:
              nr_review_count += 1
              nr_user_id.append(review['user_id'])
              if photos > 0:
                nr_photo_is += 1
                nr__photo_count += photos
              
              '''
              nr_review_count += 1
              if photos > 0:
                #nr_photo_is += 1
                nr__photo_count += photos
              print (review['user_id'])
              if review['user_id'] in user_id.keys():
                print (user_id[review['user_id']])
                #print (user_id[review['user_id']] )
                nr__photo_count = 1.0  * nr__photo_count/ user_id[review['user_id']]
                if nr__photo_count >= 1:
                  nr_photo_is+=1
                '''
          f = open('/content/drive/My Drive/Yelp/Data/statistics/nr_user_id', 'wb')
          pickle.dump(nr_user_id, f)
          f.close()

          if r_review_count > 0:
            r_photo_is_per = 1.0 * r_photo_is / r_review_count
          if nr_review_count > 0:
            nr_photo_is_per = 1.0 * nr_photo_is / nr_review_count
          photo_count.write(b_id + ' ' + str(r_review_count) + ' ' \
                 + str(r_photo_is) + ' ' + str(r_photo_count) + ' ' \
                 + str(r_photo_is_per) + ' ' \
                 + str(nr_review_count) + ' ' \
                 + str(nr_photo_is) + ' ' + str(nr__photo_count) + ' ' \
                 + str(nr_photo_is_per) + ' ' + '\n' \
                 )

pics_count()


In [0]:
#get the users of "recommended reviews"
def users_count():
 with open('/content/drive/My Drive/Yelp/Data/statistics/review_count_filtered.txt', 'r') as rev_fil:
   with open('/content/drive/My Drive/Yelp/yelp_dataset/user.json', 'r') as users:
      user_id={}
      for r in rev_fil:
        b_id = r.split()[0]
        if os.path.exists('/content/drive/My Drive/Yelp/Data/original/top1000/' + b_id +'.json'):
          with open('/content/drive/My Drive/Yelp/Data/original/top1000/'+ b_id +'.json', 'r') as biz:
            for line in biz:
              review = json.loads(line, object_pairs_hook=OrderedDict)
              user_id[review['user_id']] = 0
        for u in users:
          item = json.loads(u, object_pairs_hook=OrderedDict)
          if item['user_id'] in user_id.keys():
            user_id[item['user_id']] = item['review_count']
        f = open('/content/drive/My Drive/Yelp/Data/statistics/user_id_review_count', 'wb')
        pickle.dump(user_id, f)
        f.close()

users_count()


In [0]:
def users_count():
 with open('/content/drive/My Drive/Yelp/Data/statistics/review_count_filtered.txt', 'r') as rev_fil:
   with open('/content/drive/My Drive/Yelp/yelp_dataset/user.json', 'r') as users:
      with open('/content/drive/My Drive/Yelp/Data/statistics/users.txt', 'w') as users_target:
        user_id={}
        for r in rev_fil:
          b_id = r.split()[0]
          if os.path.exists('/content/drive/My Drive/Yelp/Data/scraped/' + b_id +'.json'):
            with open('/content/drive/My Drive/Yelp/Data/scraped/'+ b_id +'.json', 'r') as biz:
              for line in biz:
                review = json.loads(line, object_pairs_hook=OrderedDict)
                if review['label'] == 0:
                  user_id[review['user_id']] = 0
        for u in users:
          item = json.loads(u, object_pairs_hook=OrderedDict)
          if item['user_id'] in user_id.keys():
            user_id[item['user_id']] = item['review_count']
        f = open('/content/drive/My Drive/Yelp/Data/statistics/user_id_review_count2', 'wb')
        pickle.dump(user_id, f)
        f.close()

users_count()


In [0]:

f = open('/content/drive/My Drive/Yelp/Data/statistics/user_id_review_count2', 'rb')
u  = pickle.load(f)
print (len(u.keys()))
'''
with open('/content/drive/My Drive/Yelp/yelp_dataset/user.json', 'r') as users:
  for idx, line in enumerate(users):
    
    l = json.loads(line, object_pairs_hook=OrderedDict)
    if l['user_id'] in u.keys():
      print (l)
'''

11262


"\nwith open('/content/drive/My Drive/Yelp/yelp_dataset/user.json', 'r') as users:\n  for idx, line in enumerate(users):\n    \n    l = json.loads(line, object_pairs_hook=OrderedDict)\n    if l['user_id'] in u.keys():\n      print (l)\n"

In [0]:
f = open('/content/drive/My Drive/Yelp/Data/statistics/user_id_review_count', 'rb')
u  = pickle.load(f)
print (u)
print (len(u.keys()))

{'2K62MJ4CJ19L8Tp5pRfjfQ': 57, 'A0kENtCCoVT3m7T35zb2Vg': 348, 'SuOLY03LW5ZcnynKhbTydA': 105, 'lymyUak6KNcNKoDbK87MiQ': 1, '6vU0I5XgCv9OQHZ76rV6qw': 6, 'nyL_LfmAaf15o8aj7WHnPw': 4, '-ga7pQvnJcMB1_pIapHQRQ': 73, 'agqWketq-FhYwVmRyli4jA': 73, '02AgiiwLg_XAhkvG-wGZKQ': 146, 'paYZw5wsS25IzzBZtijB8A': 96, 'nCqOSH2I6-k93B04FwouIA': 131, 'U-L3aXEkYzDX5J8w8lLbCQ': 31, 'R-xGsTpwlwuOe_vAbg_aeA': 237, 'Y598FH7fNMq45DkVSbN0DA': 36, 'KmHtoogKhK-ID7FWGOPvag': 144, '7szRk9_Fs68aJrb04r-sbQ': 25, 'o_DeoZ0Ogu32D18cgEDkbA': 76, '7DBDjEN6SnVgLe5zWOe-FA': 60, 'U6m0prDFGql_LkXredF0-A': 11, '79Oujo22kxeFf_d9sz1F3g': 10, 'fQk7tJKxXhc9S8vEZwRYCA': 50, 'g9xWuPtBrZ7ajjESXHgQYg': 120, 'ytEi22nfhGNHZxfzLRRj3w': 4, '0mjFS4eWKJCYwBwO7DU8jw': 49, 'QWSiTs6VS6BDraj-j9huAg': 84, 'PkRFSQgSfca9Tamq7b2LdQ': 69, 'yCtAc7BXSYdHWQYSvsGrFw': 19, 'zWWcik1fRPZviBCQLC26FQ': 602, '9R_WvU71wwV9mMSHhATfbg': 2, 'kxXr01bsJDQ_OdNWVZafGA': 12, '_1Ol88yKJobuU81JElTlaw': 32, 'dQF4hwQRJXCcSKCTx2CReQ': 13, 'zkwsOfvvbAm_7nQaFy2kTA': 9, 'skxAQW

In [0]:
f = open('/content/drive/My Drive/Yelp/Data/statistics/user_id_review_count', 'rb')
u  = pickle.load(f)
uids =tuple([uid for uid in u.keys()])
with open('/content/drive/My Drive/Yelp/yelp_dataset/user.json', 'r') as users:
  for line in users:
    user = json.loads(line, object_pairs_hook=OrderedDict)
    if user['user_id'] in uids:
        #print (user['friends'])


'''
G=nx.Graph()
G.add_edge(2,3)
G.add_edge(3,1)
G.add_edge(2,1)
G.add_edge(4,1)
G.add_edge(5,1)
nodes  = G.nodes()

degree = G.degree()
size = [50*(degree[i]+1.0) for i in nodes]
pos=nx.spring_layout(G)
nx.draw(G,nodelist=nodes,  node_color = 'red', with_labels=False, cmap=plt.cm.Reds, pos=pos, node_size=size, arrows=True)
plt.show()
'''


In [0]:
G=nx.Graph()
G.add_edge(2,3)
G.add_edge(3,1)
G.add_edge(2,1)
G.add_edge(4,1)
G.add_edge(5,1)
nodes  = G.nodes()

degree = G.degree()
size = [50*(degree[i]+1.0) for i in nodes]
pos=nx.spring_layout(G)
nx.draw(G,nodelist=nodes,  node_color = 'red', with_labels=False, cmap=plt.cm.Reds, pos=pos, node_size=size, arrows=True)
plt.show()